# Tools  

At the end of the day, what good is an LLM if I have to read its output and manually perform some action? We want the LLM to have a real-world impact—to produce actual side effects. That is to say:  

- Place an order  
- Send an email  
- Make an HTTP request  
- Query a database  
- And more...  

All without requiring human intervention.

### Example Single tool (Weather)

In [48]:
import anthropic
from utils import ANTHROPIC_API_KEY

client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)

In [49]:
def query_w_tool(msg, tools, process_f):
    model = "claude-3-5-haiku-latest"
    print(f"\n{'='*50}\nUser Message: {msg}\n{'='*50}")
    message = client.messages.create(
        model=model,
        max_tokens=4096,
        messages=[{"role": "user", "content": msg}],
        tools=tools,
        tool_choice={"type": "auto"}
    )

    print(f"\nInitial Response:")
    print(f"Stop Reason: {message.stop_reason}")
    print(f"Content: {message.content}")

    seen_msgs = []

    if message.stop_reason == "tool_use":
        tool_use = next(block for block in message.content if block.type == "tool_use")
        tool_name = tool_use.name
        tool_input = tool_use.input

        print(f"\nTool Used: {tool_name}")
        print(f"Tool Input: {tool_input}")
        
        tool_result = process_f(tool_name, tool_input)
        print(f"Tool Result: {tool_result}")

        response = client.messages.create(
            model=model,
            max_tokens=4096,
            messages=[
                {"role": "user", "content": msg},
                {"role": "assistant", "content": message.content},
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "tool_result",
                            "tool_use_id": tool_use.id,
                            "content": tool_result,
                        }
                    ],
                },
            ],
            tools=tools,
        )
    else:
        response = message

    final_response = next((block.text for block in response.content if hasattr(block, "text")), None)
    print(response.content)
    print(f"\nFinal Response: {final_response}")
    return final_response


In [50]:
def get_weather(location: str, unit: str = "celsius"):
    if unit == "celsius": return "23 deg C"
    return "67 F"

weather_tool = {
  "name": "get_weather",
  "description": "Get the current weather in a given location",
  "input_schema": {
    "type": "object",
    "properties": {
      "location": {
        "type": "string",
        "description": "The city and state, e.g. San Francisco, CA"
      },
      "unit": {
        "type": "string",
        "enum": ["celsius", "fahrenheit"],
        "description": "The unit of temperature, either 'celsius' or 'fahrenheit'"
      }
    },
    "required": ["location"]
  }
}

def process(name, input_obj):
    if name == "get_weather":
        return get_weather(input_obj["location"], input_obj["unit"])

query_w_tool("Get weather in calagry in metric unit", [weather_tool], process)


User Message: Get weather in calagry in metric unit

Initial Response:
Stop Reason: tool_use
Content: [TextBlock(citations=None, text="I'll help you get the weather for Calgary using the metric (Celsius) unit. I'll use the get_weather function to retrieve this information.", type='text'), ToolUseBlock(id='toolu_01MisxMaPTJGXgYkuy2TZVQr', input={'location': 'Calgary, AB', 'unit': 'celsius'}, name='get_weather', type='tool_use')]

Tool Used: get_weather
Tool Input: {'location': 'Calgary, AB', 'unit': 'celsius'}
Tool Result: 23 deg C
[TextBlock(citations=None, text='The current temperature in Calgary is 23 degrees Celsius. It seems like a pleasant day! Is there anything else you would like to know about the weather?', type='text')]

Final Response: The current temperature in Calgary is 23 degrees Celsius. It seems like a pleasant day! Is there anything else you would like to know about the weather?


'The current temperature in Calgary is 23 degrees Celsius. It seems like a pleasant day! Is there anything else you would like to know about the weather?'

### Quiz 

How would you handle tooling if it was not offered by the LLM provider?  

--------  

Now, almost all providers offer first-class tool support. Previously, we had to write a system prompt that defined tool attributes and tool parameters. Additionally, we had to write a parser to process the LLM output and extract tool calls and input values.  

Now, it's much more straightforward. If you're working with a custom model, you can use the system prompt to define possible tools and the format for tool calls.  

```  
You are an intelligent assistant. You may request a tool...  
....  
....  

<tools>  
<tool name="">  
<arg name="", description="">  
</tool>  

{USER PROMPT}  
```

### Quiz (Calculator)

In [39]:
import re

def simple_calculator(expression):
    expression = re.sub(r'[^0-9+\-*/().]', '', expression)
    try: return str(eval(expression))
    except (SyntaxError, ZeroDivisionError, NameError, TypeError, OverflowError): return "Error: Invalid expression"

In [44]:
simple_calculator("1*2+2")

'4'

In [ ]:
calculator_tool = {
    "name": "simple_calculator",
    "description": "",
    "input_schema": {} # FILL ME
}

def process(tool_name, tool_input):
    if tool_name == "simple_calculator":
        return "" # FILL ME

query_w_tool("what's 1*1 + 2*2?", [calculator_tool], process)

### Quiz (Simple transaction bot)

In [47]:
USERS = {
    "helena": {
        "name": "Helena W",
        "email": "helena@uwaterloo.ca",
        "phone": "226-110-1110"
    },
    "jeremy": {
        "name": "Jeremy A",
        "email": "jeremy@uwaterloo.ca",
        "phone": "226-120-1220"
    }
}

BOOKINGS = {
    "b1": {
        "id": "b1",
        "title": "Breakfast at Hotel Pkg",
        "price": 20.00,
        "status": "booked",
    },
    "b2": {
        "id": "b2",
        "title": "Dinner at Hotel Pkg",
        "price": 120.00,
        "status": "booked",
    }
}

def get_user(user_id: str):
    return USERS.get(user_id, "Customer not found")
    return {
        "name": "Helena W",
        "age": 25,
        "bookings": [11, 12]
    }

def get_booking_detail(booking_id: str):
    return BOOKINGS.get(booking_id, "Booking not found")

def cancel_booking_detail(booking_id: str):
    if booking_id in BOOKINGS:
        return True # fake delete
    return False

In [ ]:
info_tool = {
    "name": "get_user",
    "description": "", # Fill me
    "input_schema": {} # Fill me
}

booking_detail_tool = {
    "name": "booking_detail_tool",
    "description": "", # Fill me
    "input_schema": {} # Fill me
}

cancel_booking_tool = {
    "name": "cancel_booking_tool",
    "description": "", # Fill me
    "input_schema": {} # Fill me
}

transaction_tools = [info_tool, booking_detail_tool, cancel_booking_tool]

def process(tool_name, tool_input):
    if tool_name == "get_user": 
        return info(tool_input["user_id"])
    elif tool_name == "get_order_details": 
        # FILL ME
        return
    elif tool_name == "cancel_order":
        return cancel_order(cancel_booking_detail["booking_id"])

In [41]:
# query_w_tool("Tell me email address for user helena?", transaction_tools, process)
# query_w_tool("Tell me phone number for user jeremy?", transaction_tools, process)
# query_w_tool("Status of booking b1", transaction_tools, process)
# query_w_tool("Cancel booking b1", transaction_tools, process)

### Quiz

**Why doesn't following work?** How would we change `query_w_tool` function?

In [43]:
def get_time(city: str):
    return "12pm"

time_tool = {
  "name": "get_time",
  "description": "Get the current time in a given location",
  "input_schema": {
    "type": "object",
    "properties": {
      "location": {
        "type": "string",
        "description": "The city and state, e.g. San Francisco, CA"
      },
    },
    "required": ["location"]
  }
}

def process(name, input_obj):
    if name == "get_time":
        return get_time(input_obj["location"], input_obj["unit"])
    if name == "get_weather":
        return get_weather(input_obj["location"], input_obj["unit"])

# query_w_tool("tell me time and temp in calagry rn", [weather_tool, time_tool], process)